In [81]:
import pandas as pd
import requests as r
from io import BytesIO

In [82]:
def query(q,url='http://127.0.0.1:8123'):
    return r.post(url=url,
            data=q.encode('utf-8'))
    

In [83]:
df = pd.read_json('data/data_save.txt',lines=True,chunksize=100000)

In [90]:
gen = (i for i in df)

In [99]:
gen.__next__().query('e!="depthUpdate"').dropna(axis=1).dtypes

e     object
E      int64
s     object
t    float64
p    float64
q    float64
b     object
a     object
T    float64
m    float64
M    float64
dtype: object

In [97]:
query('''CREATE TABLE trade_data (

                            e String, 
                            E UInt64, 
                            s String, 
                            b UInt32, 
                            a UInt32, 
                            t Float64, 
                            p Float64, 
                            q Float64, 
                            T Float64, 
                            m UInt8, 
                            M Float64

                            ) 
engine = MergeTree
ORDER BY E
''').text

''

In [95]:
gen.__next__().query('e=="depthUpdate"').dropna(axis=1).dtypes

e     object
E      int64
s     object
b     object
a     object
U    float64
u    float64
dtype: object

In [101]:
query('''CREATE TABLE du_data (

                            e     String,
                            E     UInt64,
                            s     String,
                            b     String,
                            a     String,
                            U    Float64,
                            u    Float64

                            ) 
engine = MergeTree
ORDER BY E
''').text

''

In [102]:
for data in df:
    trade=data.query('e!="depthUpdate"').dropna(axis=1)
    du=data.query('e=="depthUpdate"').dropna(axis=1)
    sample = ','.join(tuple(trade.apply(lambda x: str(tuple(x)),axis=1).to_list()))
    sample2 = ','.join(tuple(du.apply(lambda x: str(tuple(x)),axis=1).to_list()))
    query(f'INSERT INTO trade_data VALUES {sample}')
    query(f'INSERT INTO du_data VALUES {sample2}')